In [1]:
import numpy as np
import pandas as pd
import re
import time


In [2]:
df = pd.read_csv('1. product_laptop.csv')
df.head(1)

,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,Bought L Month,Product Link,Customer Say,Insight,Individual Rating,Review Title,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes
0,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...","89,990.",4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,NaN,NaN,5.0,Fast and Powerful!,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,I recently purchased the MacBook M4 with 512GB...,38


### Data Preprocessing

In [4]:
df.shape

(5768, 20)

In [5]:
df.duplicated().sum()

12

In [6]:
df[df.duplicated(keep=False)]

,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,Bought L Month,Product Link,Customer Say,Insight,Individual Rating,Review Title,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes
104,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...","47,390.",3.9,609,165,1K+,https://www.amazon.in/Lenovo-IdeaPad-i5-12450H...,Customers find the laptop's build quality good...,Build quality|Speed|Compactness|Display qualit...,5.0,Good laptop,Reviewed in India on 29 October 2024,Colour: 12th Gen i5 | 16GB | 512GB SSD,True,Like fast performance,0
106,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...","47,390.",3.9,609,165,1K+,https://www.amazon.in/Lenovo-IdeaPad-i5-12450H...,Customers find the laptop's build quality good...,Build quality|Speed|Compactness|Display qualit...,5.0,Good laptop,Reviewed in India on 29 October 2024,Colour: 12th Gen i5 | 16GB | 512GB SSD,True,Like fast performance,0
204,B0D3HG5CMG,9D3M8PA,HP,Laptop,"HP 15, AMD Ryzen 3 7320U (8GB LPDDR5, 512GB SS...","28,990.",4.0,"1,653",355,1K+,https://www.amazon.in/HP-Laptop-15-6-inch-Grap...,"Customers find the laptop performs well, with ...",Value for money|Speed|Performance|Design|Good ...,3.0,Misleading description in specs,Reviewed in India on 6 January 2024,Colour: Natural Silver,True,Descrition stating Preloaded with Windows 11 &...,5
206,B0D3HG5CMG,9D3M8PA,HP,Laptop,"HP 15, AMD Ryzen 3 7320U (8GB LPDDR5, 512GB SS...","28,990.",4.0,"1,653",355,1K+,https://www.amazon.in/HP-Laptop-15-6-inch-Grap...,"Customers find the laptop performs well, with ...",Value for money|Speed|Performance|Design|Good ...,3.0,Misleading description in specs,Reviewed in India on 6 January 2024,Colour: Natural Silver,True,Descrition stating Preloaded with Windows 11 &...,5
352,B08N5W4NNB,MGN63HN/A,Apple,Laptop,"Apple MacBook Air Laptop: Apple M1 chip, 13.3-...","54,990.",4.6,"7,004","1,500",3K+,https://www.amazon.in/Apple-MacBook-Chip-13-in...,Customers find this MacBook Air to be an excel...,Quality|Value for money|Performance|Build qual...,5.0,Exactly what I needed,Reviewed in India on 3 January 2025,Colour: Space Grey|Size: 256 GB,True,Sound quality is very good I usually use it at...,2
360,B08N5W4NNB,MGN63HN/A,Apple,Laptop,"Apple MacBook Air Laptop: Apple M1 chip, 13.3-...","54,990.",4.6,"7,004","1,500",3K+,https://www.amazon.in/Apple-MacBook-Chip-13-in...,Customers find this MacBook Air to be an excel...,Quality|Value for money|Performance|Build qual...,5.0,Exactly what I needed,Reviewed in India on 3 January 2025,Colour: Space Grey|Size: 256 GB,True,Sound quality is very good I usually use it at...,2
824,B0F4R5W1NC,fd0569TU,HP,Laptop,"HP 15, 13th Gen Intel Core i3-1315U Laptop (8G...","34,990.",4.1,"2,012",338,100+,https://www.amazon.in/HP-i3-1315U-Anti-Glare-M...,Customers find this laptop to be a good value ...,Quality|Speed|Value for money|Compactness|Use|...,5.0,Highly recommended product,Reviewed in India on 1 October 2023,Colour: Natural Silver,True,"I am very happy with this model. System, batte...",10
833,B0F4R5W1NC,fd0569TU,HP,Laptop,"HP 15, 13th Gen Intel Core i3-1315U Laptop (8G...","34,990.",4.1,"2,012",338,100+,https://www.amazon.in/HP-i3-1315U-Anti-Glare-M...,Customers find this laptop to be a good value ...,Quality|Speed|Value for money|Compactness|Use|...,5.0,Highly recommended product,Reviewed in India on 1 October 2023,Colour: Natural Silver,True,"I am very happy with this model. System, batte...",10
1674,B0D872NJSM,15IAU7,Lenovo,Laptop,Lenovo IdeaPad 3 12th Gen Intel Core i3-1215U ...,"34,539.",4.1,"1,145",208,50+,https://www.amazon.in/Lenovo-IdeaPad-i3-1215U-...,Customers find the laptop excellent for educat...,Functionality|Value for money|Speed|Use|Qualit...,4.0,Good product,Reviewed in India on 19 January 2023,Style Name: Intel Core i3-11th Gen,True,Good,0
1677,B0D872NJSM,15IAU7,Lenovo,Laptop,Lenovo IdeaPad 3 12th Gen Intel Core

In [7]:
df.drop_duplicates(inplace=True)

In [8]:
df.shape

(5756, 20)

In [9]:
df.duplicated(subset=["Individual Rating", "Review Title", "Review Date", "Product Description", 'Review Text'], keep=False).sum()

2213

In [10]:
df[df.duplicated(subset=["Individual Rating", "Review Title", "Review Date", "Product Description", 'Review Text'], keep=False)] \
.sort_values(by=["Review Title", "Review Date", "Product Description", 'Review Text']).head(10)

,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,Bought L Month,Product Link,Customer Say,Insight,Individual Rating,Review Title,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes
4933,B0D6G4CVZ7,MediaTek Helio G99 (Octa-Core),Lenovo,Tablet,Lenovo Tab Plus with Octa JBL Hi-Fi Speakers| ...,"15,990.",4.4,822,317,1K+,https://www.amazon.in/Lenovo-Speakers-Refresh-...,"Customers praise the tablet's sound quality, w...",Sound quality|Quality|Display quality|Value fo...,5.0,.,Reviewed in India on 10 June 2025,Size: 8GB RAM + 128GB ROM,True,/,3
5307,B0D6G4SKGK,MediaTek Helio G99 (Octa-Core),Lenovo,Tablet,Lenovo {Smartchoice) Tab Plus with Octa JBL Hi...,"17,999.",4.4,822,317,1K+,https://www.amazon.in/Lenovo-Speakers-Refresh-...,"Customers praise the tablet's sound quality, w...",Sound quality|Quality|Display quality|Value fo...,5.0,.,Reviewed in India on 10 June 2025,Size: 8GB RAM + 128GB ROM,True,/,3
419,B0F4R3GFMQ,fd0573TU,HP,Laptop,"HP 15, 13th Gen Intel Core i3-1315U (12GB DDR4...","35,990.",3.7,147,61,1K+,https://www.amazon.in/HP-i3-1315U-Anti-Glare-M...,Customers find the laptop offers good value fo...,Value for money|Performance|Sound quality|Lapt...,5.0,.,Reviewed in India on 28 May 2025,Size: 12GB RAM with MS Office 2024,True,NaN,One
1189,B0D2KDCWJL,91Q92PA,HP,Laptop,"HP 15, 13th Gen Intel Core i3-1315U Laptop (8G...","33,990.",3.7,147,61,300+,https://www.amazon.in/HP-i3-1315U-Anti-Glare-M...,Customers find the laptop offers good value fo...,Value for money|Performance|Sound quality|Lapt...,5.0,.,Reviewed in India on 28 May 2025,Size: 12GB RAM with MS Office 2024,True,NaN,One
1350,B0F4R6GXWJ,fd0572TU,HP,Laptop,"HP 15, 13th Gen Intel Core i3-1315U Laptop (8G...","34,990.",3.7,147,61,100+,https://www.amazon.in/HP-i3-1315U-Anti-Glare-M...,Customers find the laptop offers good value fo...,Value for money|Performance|Sound quality|Lapt...,5.0,.,Reviewed in India on 28 May 2025,Size: 12GB RAM with MS Office 2024,True,NaN,One
4004,B0FG3LB357,15IAX9E,Lenovo,Laptop,Lenovo LOQ Essential Intel Core i5-12450HX| NV...,"62,490.",3.7,50,25,NaN,https://www.amazon.in/Lenovo-Essential-i5-1245...,Customers find the laptop's quality very good....,Quality|Battery life,5.0,10/10 laptop,Reviewed in India on 27 May 2025,Colour: Luna Grey,True,Best product ever should buy it,0
5471,B0D49RN3X6,15IAX9,Lenovo,Laptop,Lenovo Smartchoice LOQ 12th Gen Intel Core i5-...,"65,990.",3.7,50,25,50+,https://www.amazon.in/Lenovo-i5-12450HX-300Nit...,Customers find the laptop's quality very good....,Quality|Battery life,5.0,10/10 laptop,Reviewed in India on 27 May 2025,Colour: Luna Grey,True,Best product ever should buy it,0
396,B0F2162VGQ,15IRH10,Lenovo,Laptop,Lenovo Smartchoice Ideapad Slim 3 13Th Gen Int...,"61,990.",3.8,42,30,1K+,https://www.amazon.in/Lenovo-Smartchoice-I7-13...,Customers find the laptop to be good value for...,Laptop quality|Value for money|Build quality|P...,1.0,100 to 60% in 10 min,Reviewed in India on 5 May 2025,Style Name: i7-13620H,True,🔋 back is very bad . Rest is ok,8
4336,B0F637DPFW,15IRH10,Lenovo,Laptop,Lenovo IdeaPad Slim 3 13th Gen Intel Core i5-1...,"57,990.",3.8,42,30,100+,https://www.amazon.in/Lenovo-IdeaPad-i5-13420H...,Customers find the laptop to be good value for...,Laptop quality|Value for money|Build quality|P...,1.0,100 to 60% in 10 min,Reviewed in India on 5 May 2025,Style Name: i7-13620H,True,🔋 back is very bad . Rest is ok,8
87,B0DP7BZ8FD,14IAH8,Lenovo,Laptop,"Lenovo IdeaPad Slim 3, Intel Core i5-12450H, 1...","47,390.",3.9,609,165,1K+,https://www.amazon.in/Lenovo-IdeaPad-i5-12450H...,Customers find the laptop's build quality good...,Build quality|Speed|Compactness|Display qualit...,5.0,14 inch laptop this laptop are very compact in...,Reviewed in India on 2 October 2024,Colour: 12th Gen i5 | 16GB | 512GB SSD,True,/,4


In [11]:
df.drop_duplicates(subset=["Individual Rating", "Review Title", "Review Date", "Product Description", 'Review Text'], keep='first', inplace=True)

In [12]:
df.shape

(4459, 20)

In [13]:
df.isnull().sum()

ASIN                      0
Model Number              0
Manufacturer Name         0
Generic Name             70
Product Title             0
Price                     0
Overall Rating            1
Rating Count              1
Review Count              1
Bought L Month         1354
Product Link              0
Customer Say            603
Insight                 603
Individual Rating         2
Review Title             11
Review Date               1
Product Description     816
Verified Purchase         0
Review Text              84
Helpful Votes             1
dtype: int64

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4459 entries, 0 to 5767
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ASIN                 4459 non-null   object 
 1   Model Number         4459 non-null   object 
 2   Manufacturer Name    4459 non-null   object 
 3   Generic Name         4389 non-null   object 
 4   Product Title        4459 non-null   object 
 5   Price                4459 non-null   object 
 6   Overall Rating       4458 non-null   float64
 7   Rating Count         4458 non-null   object 
 8   Review Count         4458 non-null   object 
 9   Bought L Month       3105 non-null   object 
 10  Product Link         4459 non-null   object 
 11  Customer Say         3856 non-null   object 
 12  Insight              3856 non-null   object 
 13  Individual Rating    4457 non-null   float64
 14  Review Title         4448 non-null   object 
 15  Review Date          4458 non-null   object

In [15]:
df['Price'].nunique()

147

In [16]:
df['Price'].unique()

array(['89,990.', '68,990.', '47,390.', '28,990.', '22,220.', '46,990.',
       '54,990.', '59,090.', '61,990.', '1,47,990.', '35,990.', '51,990.',
       '36,990.', '33,890.', '33,990.', '22,390.', '74,990.', '80,990.',
       '77,990.', '34,990.', '25,990.', '26,490.', '33,280.', '42,990.',
       '27,999.', '25,490.', '64,990.', '21,890.', '13,990.', '42,999.',
       '30,990.', '3,26,611.', '34,539.', '35,202.', '62,990.', '26,990.',
       '29,101.', '1,29,990.', '2,39,990.', '25,500.', '1,95,805.',
       '39,490.', '41,490.', '33,490.', '1,89,990.', '22,440.',
       '1,20,990.', '30,650.', '28,490.', '2,92,990.', '24,990.',
       '27,990.', '31,490.', '31,251.', '42,490.', '1,99,999.',
       '1,62,990.', '32,900.', '2,11,990.', '1,35,900.', '35,690.',
       '1,52,991.', '4,74,490.', '1,99,990.', '62,985.', '2,01,990.',
       '1,92,140.', '1,15,990.', '48,053.', '1,27,990.', '1,08,940.',
       '90,990.', '98,990.', '84,990.', '1,09,990.', '99,990.', '65,970.',
       '94,99

In [17]:
df['Price'] = df['Price'].str.replace(",", "")
df['Price'] = df['Price'].str.strip('.')
df['Price'] = df['Price'].astype(dtype = int)
df['Price'].head(2)

0    89990
1    89990
Name: Price, dtype: int32

In [18]:
df['Rating Count'].nunique()

90

In [19]:
df['Rating Count'].unique()

array(['33', nan, '609', '1,653', '9', '60', '7,004', '4', '42', '6',
       '147', '26', '2,057', '82', '171', '25', '7', '50', '2,012', '73',
       '13', '1', '1,128', '23', '233', '196', '522', '10', '285', '2',
       '1,145', '12', '169', '18', '14', '15', '3', '817', '320', '244',
       '72', '19', '535', '40', '74', '24', '189', '45', '17', '235',
       '178', '29', '41', '141', '86', '16', '20', '5', '47', '56', '575',
       '11', '1,049', '117', '27', '37', '59', '69', '61', '1,495',
       '1,441', '8', '360', '58', '102', '131', '49', '238', '1,164',
       '468', '822', '110', '54', '21', '52', '137', '30', '261', '225',
       '36', '76'], dtype=object)

In [20]:
df['Rating Count'] = df['Rating Count'].str.replace(",", "")
df['Rating Count'] = df['Rating Count'].astype(dtype = 'Int64')
df['Rating Count'].head(2)

0    33
1    33
Name: Rating Count, dtype: Int64

In [21]:
df['Review Count'].nunique()

69

In [22]:
df['Review Count'].unique()

array(['15', nan, '165', '355', '7', '42', '1,500', '4', '30', '3', '61',
       '18', '431', '46', '48', '14', '2', '338', '43', '13', '1', '234',
       '11', '101', '102', '65', '135', '136', '68', '208', '9', '70',
       '230', '231', '74', '8', '94', '25', '12', '5', '138', '10', '89',
       '20', '90', '21', '33', '6', '23', '173', '291', '45', '19', '22',
       '26', '370', '331', '17', '56', '47', '76', '387', '127', '317',
       '34', '31', '58', '51', '82', '36'], dtype=object)

In [23]:
df['Review Count'] = df['Review Count'].str.replace(",", "")
df['Review Count'] = df['Review Count'].astype(dtype = 'Int64')
df['Review Count'].head(2)

0    15
1    15
Name: Review Count, dtype: Int64

In [24]:
df['Bought L Month'].unique()

array(['300+', nan, '1K+', '200+', '500+', '3K+', '400+', '100+', '50+'],
      dtype=object)

In [25]:
df['Helpful Votes'].nunique()

89

In [26]:
df['Helpful Votes'].unique()

array(['38', '118', '4', 'One', '25', '35', '0', '5', '10', '3', '2', nan,
       '9', '99', '12', '21', '14', '6', '15', '26', '16', '7', '20', '8',
       '18', '11', '13', '82', '34', '44', '28', '22', '33', '42', '19',
       '229', '24', '195', '94', '178', '68', '113', '17', '137', '27',
       '45', '95', '90', '189', '316', '39', '41', '53', '23', '117',
       '29', '281', '87', '49', '51', '31', '75', '37', '47', '48', '32',
       '40', '36', '79', '30', '147', '67', '58', '66', '43', '81', '142',
       '96', '76', '77', '80', '84', '165', '518', '73', '140', '230',
       '50', '64', '65'], dtype=object)

In [27]:
def func(x):
    if isinstance(x, str) and x.lower()=='one':
        return '1'
    else:
        return x

df['Helpful Votes'] = df['Helpful Votes'].apply(func)

In [28]:
df['Helpful Votes'].unique()

array(['38', '118', '4', '1', '25', '35', '0', '5', '10', '3', '2', nan,
       '9', '99', '12', '21', '14', '6', '15', '26', '16', '7', '20', '8',
       '18', '11', '13', '82', '34', '44', '28', '22', '33', '42', '19',
       '229', '24', '195', '94', '178', '68', '113', '17', '137', '27',
       '45', '95', '90', '189', '316', '39', '41', '53', '23', '117',
       '29', '281', '87', '49', '51', '31', '75', '37', '47', '48', '32',
       '40', '36', '79', '30', '147', '67', '58', '66', '43', '81', '142',
       '96', '76', '77', '80', '84', '165', '518', '73', '140', '230',
       '50', '64', '65'], dtype=object)

In [29]:
df['Helpful Votes'] = df['Helpful Votes'].astype(dtype="Int64")
df['Helpful Votes']

0        38
1       118
2         4
3         1
4        25
       ... 
5763      0
5764      1
5765      0
5766      1
5767      1
Name: Helpful Votes, Length: 4459, dtype: Int64

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4459 entries, 0 to 5767
Data columns (total 20 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   ASIN                 4459 non-null   object 
 1   Model Number         4459 non-null   object 
 2   Manufacturer Name    4459 non-null   object 
 3   Generic Name         4389 non-null   object 
 4   Product Title        4459 non-null   object 
 5   Price                4459 non-null   int32  
 6   Overall Rating       4458 non-null   float64
 7   Rating Count         4458 non-null   Int64  
 8   Review Count         4458 non-null   Int64  
 9   Bought L Month       3105 non-null   object 
 10  Product Link         4459 non-null   object 
 11  Customer Say         3856 non-null   object 
 12  Insight              3856 non-null   object 
 13  Individual Rating    4457 non-null   float64
 14  Review Title         4448 non-null   object 
 15  Review Date          4458 non-null   object

In [31]:
df['ASIN'].nunique()

185

In [32]:
df['ASIN'].unique()

array(['B0DZDC247V', 'B0F8P4Y7VF', 'B0DP7BZ8FD', 'B0D3HG5CMG',
       'B0FCD9LKLZ', 'B0F32ZBDNJ', 'B08N5W4NNB', 'B0DZJ262NX',
       'B0FFGWWHBP', 'B0F2162VGQ', 'B0DLHFS9L6', 'B0F4R3GFMQ',
       'B0F4R6H3NB', 'B0F8MX6QK1', 'B0F29HNJL1', 'B0C4ZM63RP',
       'B0DV984J97', 'B0D9VLF328', 'B0DLHGQ17K', 'B0DLHG997J',
       'B0DLHFM2XL', 'B0F4R5W1NC', 'B0DVGQPW13', 'B0F6TJ164J',
       'B0FCMR6PK3', 'B09RZRF1PJ', 'B0DVLPPPRN', 'B0DZCB3G9Q',
       'B0CL7CMTXS', 'B0F66VQB4S', 'B0C28FKJXG', 'B0F2TMZJ24',
       'B0CFLMMWY2', 'B0F6DCF3VS', 'B0BWK3TS1C', 'B0D872NJSM',
       'B0F5HM9J6S', 'B0CJ2KWNNQ', 'B0DTTS34NW', 'B0CSFW9D4W',
       'B0CSKDDGPG', 'B0DZ6K1PHP', 'B0DPX6HWM3', 'B0BQJ9HZPP',
       'B0DLHTDZVL', 'B0DRCTY5JH', 'B0CP615Z5M', 'B0D86XBRVW',
       'B0CD4BQ4SF', 'B0CY2PLQ8N', 'B0F8WC83WT', 'B0DP9TGR7D',
       'B0F6LK392D', 'B0DZDDPXY5', 'B0F6TKGHLG', 'B0B4JPC8GT',
       'B0C66685GZ', 'B0DLHGCB3C', 'B09QQM3J65', 'B0F6LTFFPM',
       'B0DFH99H7X', 'B0845NYSGW', 'B0D39BVJ7Y', 'B0CXX

In [33]:
## ASIN column is clean

In [34]:
df['Generic Name'].nunique()

21

In [35]:
df['Generic Name'].unique()

array(['MacBook Air', 'Laptop', 'Business Laptop', 'Notebook Laptop',
       '13-inch MacBook Air (M3, 2024)', 'MacBook Air 13-inch (M2, 2022)',
       'BUSINESS LAPTOP', 'HP Laptop 15s-eq2144AU', 'Tablet', nan,
       'Chromebook', 'LAPTOP', '16-inch MacBook Pro (M4 Pro, 2024)',
       '14-inch MacBook Pro (M4 Max, 2024)', 'Chromebook x360',
       '14-inch MacBook Pro (M4, 2024)', 'Gaming Laptop',
       '15-inch MacBook Air (M3, 2024)', 'Laptopo',
       '16-inch MacBook Pro (M2 Max, 2023)', 'laptop',
       'Laptop, Adapter. User manual'], dtype=object)

In [36]:
df[(df['Generic Name'].str.contains('laptop', case = False, na=False)) | (df['Generic Name'].str.contains('macbook', case = False, na=False))]

,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,Bought L Month,Product Link,Customer Say,Insight,Individual Rating,Review Title,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes
0,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,NaN,NaN,5.0,Fast and Powerful!,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,I recently purchased the MacBook M4 with 512GB...,38
1,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,NaN,NaN,5.0,Elevate Your Productivity with the Perfect Laptop,Reviewed in India on 4 April 2025,Colour: Midnight|Size: 16GB Unified Memory|Sty...,True,I placed the order on March 28th and received ...,118
2,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,NaN,NaN,5.0,"Great build quality, good for everyday/office use",Reviewed in India on 9 May 2025,Colour: Sky Blue|Size: 16GB Unified Memory|Sty...,True,New to the apple ecosystem. Every single apple...,4
3,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,NaN,NaN,5.0,Sleek and super fast,Reviewed in India on 4 July 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,"Very easy to use , has met my expectations",1
4,B0DZDC247V,MW0Y3HN/A,Apple,MacBook Air,"Apple 2025 MacBook Air (13-inch, Apple M4 chip...",89990,4.5,33,15,300+,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,NaN,NaN,5.0,Overwhelmed with my purchase,Reviewed in India on 19 April 2025,Colour: Starlight|Size: 16GB Unified Memory|St...,True,"Just got it 2 days ago. So far, just overwhelm...",25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5763,B0D268HBQY,16-af0015TU,HP,Laptop,"HP Pavilion, Intel Core Ultra 5-125U Ai Powere...",60990,3.4,76,36,NaN,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Customers find the laptop performs well and is...,Build quality|Performance|Display quality|Came...,5.0,"Bigger, Brighter & smooth experience....",Reviewed in India on 12 November 2024,Colour: Ultra 5,False,I brought 53720 with offer in Diwali time...Ca...,0
5764,B0D268HBQY,16-af0015TU,HP,Laptop,"HP Pavilion, Intel Core Ultra 5-125U Ai Powere...",60990,3.4,76,36,NaN,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Customers find the laptop performs well and is...,Build quality|Performance|Display quality|Came...,5.0,Great Product,Reviewed in India on 28 July 2024,Colour: Ultra 5,False,Great product ...Love the fast processing spee...,1
5765,B0D268HBQY,16-af0015TU,HP,Laptop,"HP Pavilion, Intel Core Ultra 5-125U Ai Powere...",60990,3.4,76,36,NaN,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Customers find the laptop performs well and is...,Build quality|Performance|Display quality|Came...,1.0,Worst product never buy it,Reviewed in India on 22 November 2024,Colour: Ultra 7,False,Hp world worst product Hp support worst servic...,0
5766,B0D268HBQY,16-af0015TU,HP,Laptop,"HP Pavilion, Intel Core Ultra 5-125U Ai Powere...",60990,3.4,76,36,NaN,https://www.amazon.in/sspa/click?ie=UTF8&spc=M...,Customers find the laptop performs well and is...,Build quality|Performance|Display quality|Came...,1.0,Scratch,Reviewed in India on 24 January 2025,Colour: Ultra 7,True,NaN,1


In [37]:
df[~(df['Generic Name'].str.contains('laptop|macbook|chromebook', case = False, na=False))]

,ASIN,Model Number,Manufacturer Name,Generic Name,Product Title,Price,Overall Rating,Rating Count,Review Count,Bought L Month,Product Link,Customer Say,Insight,Individual Rating,Review Title,Review Date,Product Description,Verified Purchase,Review Text,Helpful Votes
1005,B0DZCB3G9Q,ZAE40195IN,Lenovo,Tablet,Lenovo {Smartchoice} Idea Tab Pro with Pen Plu...,27999,4.4,233,101,1K+,https://www.amazon.in/Lenovo-Smartchoice-AI-En...,Customers find this tablet to be one of the be...,Quality|Display size|Sound quality|Value for m...,5.0,The best tablet in this price range...,Reviewed in India on 4 May 2025,"Colour: Luna Grey,8 GB + 128 GB",True,It's just amazing product... No issues with it...,25
1006,B0DZCB3G9Q,ZAE40195IN,Lenovo,Tablet,Lenovo {Smartchoice} Idea Tab Pro with Pen Plu...,27999,4.4,233,101,1K+,https://www.amazon.in/Lenovo-Smartchoice-AI-En...,Customers find this tablet to be one of the be...,Quality|Display size|Sound quality|Value for m...,5.0,Wide screen and matt screen,Reviewed in India on 17 June 2025,"Colour: Anti glare Luna Grey,8 GB + 256 GB",True,Very nice product. I love the performance of i...,11
1007,B0DZCB3G9Q,ZAE40195IN,Lenovo,Tablet,Lenovo {Smartchoice} Idea Tab Pro with Pen Plu...,27999,4.4,233,101,1K+,https://www.amazon.in/Lenovo-Smartchoice-AI-En...,Customers find this tablet to be one of the be...,Quality|Display size|Sound quality|Value for m...,4.0,Very good.,Reviewed in India on 26 May 2025,"Colour: Luna Grey,12 GB + 256 GB",True,Good display but low brightness. Good audio bu...,12
1008,B0DZCB3G9Q,ZAE40195IN,Lenovo,Tablet,Lenovo {Smartchoice} Idea Tab Pro with Pen Plu...,27999,4.4,233,101,1K+,https://www.amazon.in/Lenovo-Smartchoice-AI-En...,Customers find this tablet to be one of the be...,Quality|Display size|Sound quality|Value for m...,5.0,Absolutely fabulous,Reviewed in India on 28 March 2025,"Colour: Luna Grey,12 GB + 256 GB",True,"Really great tablet.Very thin, but strong.Feel...",31
1009,B0DZCB3G9Q,ZAE40195IN,Lenovo,Tablet,Lenovo {Smartchoice} Idea Tab Pro with Pen Plu...,27999,4.4,233,101,1K+,https://www.amazon.in/Lenovo-Smartchoice-AI-En...,Customers find this tablet to be one of the be...,Quality|Display size|Sound quality|Value for m...,5.0,"Big display,good stylus,good spec, best tab in...",Reviewed in India on 22 June 2025,"Colour: Luna Grey,12 GB + 256 GB",True,"It's fantastic , I have to say it's a good alt...",8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4977,B0D6G4CVZ7,MediaTek Helio G99 (Octa-Core),Lenovo,Tablet,Lenovo Tab Plus with Octa JBL Hi-Fi Speakers| ...,15990,4.4,822,317,1K+,https://www.amazon.in/Lenovo-Speakers-Refresh-...,"Customers praise the tablet's sound quality, w...",Sound quality|Quality|Display quality|Value fo...,5.0,Nice,Reviewed in India on 5 April 2025,Size: 8GB RAM + 128GB ROM,True,/,2
4978,B0D6G4CVZ7,MediaTek Helio G99 (Octa-Core),Lenovo,Tablet,Lenovo Tab Plus with Octa JBL Hi-Fi Speakers| ...,15990,4.4,822,317,1K+,https://www.amazon.in/Lenovo-Speakers-Refresh-...,"Customers praise the tablet's sound quality, w...",Sound quality|Quality|Display quality|Value fo...,5.0,Quality,Reviewed in India on 6 April 2025,Size: 8GB RAM + 256GB ROM,True,Such a smooth product,5
4979,B0D6G4CVZ7,MediaTek Helio G99 (Octa-Core),Lenovo,Tablet,Lenovo Tab Plus with Octa JBL Hi-Fi Speakers| ...,15990,4.4,822,317,1K+,https://www.amazon.in/Lenovo-Speakers-Refresh-...,"Customers praise the tablet's sound quality, w...",Sound quality|Quality|Display quality|Value fo...,4.0,Good product,Reviewed in India on 7 November 2024,Size: 8GB RAM + 256GB ROM,True,"Back support, sound quality, grip, display is ...",2
4980,B0D6G4CVZ7,MediaTek Helio G99 (Octa-Core),Lenovo,Tablet,Lenovo Tab Plus with Octa JBL Hi-Fi Speakers| ...,15990,4.4,822,317,1K+,https://www.amazon.in/Lenovo-Speakers-Refresh-...,"Customers praise the tablet's sound quality, w...",Sound quality|Quality|Display quality|Value fo...,5.0,Bought it for reading,Reviewed in India on 28 January 202

In [38]:
df[~(df['Generic Name'].str.contains('laptop|macbook|chromebook', case = False, na=False))]['Generic Name'].nunique()

1

In [39]:
df[~(df['Generic Name'].str.contains('laptop|macbook|chromebook', case = False, na=False))]['Generic Name'].unique()

array(['Tablet', nan], dtype=object)

In [40]:
df[df['Generic Name']=="Mobile Workstation"]['Product Title'].values

array([], dtype=object)

In [41]:
df[(~df['Generic Name'].str.contains('laptop|macbook|chromebook', case = False, na=False)) & \
(~df['Product Title'].str.contains('laptop|macbook|chromebook', case = False, na=False))]['Generic Name'].nunique()

1

In [42]:
df[(~df['Generic Name'].str.contains('laptop|macbook|chromebook', case = False, na=False)) & \
(~df['Product Title'].str.contains('laptop|macbook|chromebook', case = False, na=False))]['Generic Name'].unique()

array(['Tablet'], dtype=object)

In [43]:
df.shape

(4459, 20)

In [44]:
exclude_idx = df[
    (~df['Generic Name'].str.contains('laptop|macbook|chromebook', case=False, na=False)) &
    (~df['Product Title'].str.contains('laptop|macbook|chromebook', case=False, na=False))
].index

df = df[~df.index.isin(exclude_idx)].reset_index(drop=True)


In [45]:
df.shape

(4260, 20)

In [46]:
df.isnull().sum()

ASIN                      0
Model Number              0
Manufacturer Name         0
Generic Name             70
Product Title             0
Price                     0
Overall Rating            1
Rating Count              1
Review Count              1
Bought L Month         1354
Product Link              0
Customer Say            603
Insight                 603
Individual Rating         2
Review Title             10
Review Date               1
Product Description     816
Verified Purchase         0
Review Text              78
Helpful Votes             1
dtype: int64

In [47]:
def clean_text(text):
    if pd.isna(text):
        return None

    text = re.sub(r'[^\x00-\x7F]+', '', text)

    text = re.sub(r'[^\w\s@#.,!?:/\-]', '', text)

    text = re.sub(r'\s+', ' ', text).strip()

    if len(text)<10:
        return None

    return text.lower()

df['Cleaned Review Text'] = df['Review Text'].apply(clean_text)

In [48]:
df['Cleaned Review Text'].head(2)

0    i recently purchased the macbook m4 with 512gb...
1    i placed the order on march 28th and received ...
Name: Cleaned Review Text, dtype: object

In [49]:
df.shape

(4260, 21)

In [50]:
df.isnull().sum()

ASIN                      0
Model Number              0
Manufacturer Name         0
Generic Name             70
Product Title             0
Price                     0
Overall Rating            1
Rating Count              1
Review Count              1
Bought L Month         1354
Product Link              0
Customer Say            603
Insight                 603
Individual Rating         2
Review Title             10
Review Date               1
Product Description     816
Verified Purchase         0
Review Text              78
Helpful Votes             1
Cleaned Review Text     418
dtype: int64

In [51]:
df.dropna(subset=['Cleaned Review Text'], inplace=True)

In [52]:
df.shape

(3842, 21)

In [53]:
df.isnull().sum()

ASIN                      0
Model Number              0
Manufacturer Name         0
Generic Name             56
Product Title             0
Price                     0
Overall Rating            0
Rating Count              0
Review Count              0
Bought L Month         1226
Product Link              0
Customer Say            555
Insight                 555
Individual Rating         1
Review Title              0
Review Date               0
Product Description     758
Verified Purchase         0
Review Text               0
Helpful Votes             0
Cleaned Review Text       0
dtype: int64

In [54]:
df = df.reset_index(drop=True)
df.insert(0, "Index", range(1, len(df) + 1))

In [55]:
# df.to_csv('2. Cleaned_laptop_df.csv', index=False)